In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import missingno as msn
import datetime
from pandas_profiling import ProfileReport
import sklearn

In [35]:
df = pd.read_parquet("escooter_history_2022.parquet")

In [36]:
df.loc[df.temp > 50, "temp"] = df.temp.div(100)

In [ ]:
df.registered_customer

In [37]:
df = pd.get_dummies(df)

In [38]:
df_slice2 = df[df["registered_customer"] == False].copy()

In [ ]:
df_slice2

In [44]:
df_slice3 = df_slice2.groupby(pd.Grouper(key="datetime",freq="min")).agg(Ausleihzahl=("registered_customer","count"),holiday=("holiday","mean"),workingday=("workingday","mean"),temp=("temp","mean"),atemp=("atemp","mean"),humidity=("humidity","mean"),windspeed=("windspeed","mean"),clear_few_clouds=("weather_clear, few clouds","mean"),cloudy_mist=("weather_cloudy, mist","mean"),heacy_rain_snow=("weather_heacy rain or thunderstorm or snow or ice pallets","mean"),light_snow_rain_thunderstorm=("weather_light snow or rain or thunderstorm","mean")).reset_index()

In [45]:
df_slice3

,datetime,Ausleihzahl,holiday,workingday,temp,atemp,humidity,windspeed,clear_few_clouds,cloudy_mist,heacy_rain_snow,light_snow_rain_thunderstorm
0,2020-01-04 00:31:00,1,0.0,0.0,9.84,14.395,81.0,0.0000,1.0,0.0,0.0,0.0
1,2020-01-04 00:32:00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-04 00:33:00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-04 00:34:00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-04 00:35:00,1,0.0,0.0,9.84,14.395,81.0,0.0000,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1052578,2022-01-03 23:29:00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1052579,2022-01-03 23:30:00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1052580,2022-01-03 23:31:00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1052581,2022-01-03 23:32:00,1,0.0,1.0,10.66,13.635,65.0,8.9981,1.0,0.0,0.0,0.0


In [49]:
df_slice3.dropna(inplace=True)


In [50]:
df_slice3[df_slice3.temp.isna()]

,datetime,Ausleihzahl,holiday,workingday,temp,atemp,humidity,windspeed,clear_few_clouds,cloudy_mist,heacy_rain_snow,light_snow_rain_thunderstorm


In [18]:
df_slice3.corr()

,Ausleihzahl,holiday,workingday,temp,atemp,humidity,windspeed,clear_few_clouds,cloudy_mist,heacy_rain_snow,light_snow_rain_thunderstorm
Ausleihzahl,1.000000,0.049119,-0.312724,0.459885,0.453973,-0.356967,0.091659,0.147423,-0.087559,-0.008097,-0.117013
holiday,0.049119,1.000000,-0.249638,-0.030501,-0.033990,-0.003336,0.000325,0.007435,0.006016,-0.001942,-0.023023
workingday,-0.312724,-0.249638,1.000000,0.069203,0.067394,0.000610,-0.002173,-0.044045,0.028467,-0.004074,0.031428
temp,0.459885,-0.030501,0.069203,1.000000,0.987181,-0.073364,-0.027144,0.109833,-0.082430,-0.020065,-0.058260
atemp,0.453973,-0.033990,0.067394,0.987181,1.000000,-0.055496,-0.063259,0.110350,-0.079437,-0.020238,-0.064101
humidity,-0.356967,-0.003336,0.000610,-0.073364,-0.055496,1.000000,-0.291329,-0.383379,0.228300,0.013762,0.303610
windspeed,0.091659,0.000325,-0.002173,-0.027144,-0.063259,-0.291329,1.000000,0.009674,-0.050497,-0.004503,0.066210
clear_few_clouds,0.147423,0.007435,-0.044045,0.109833,0.110350,-0.383379,0.009674,1.000000,-0.831465,-0.015553,-0.404043
cloudy_mist,-0.087559,0.006016,0.028467,-0.082430,-0.079437,0.228300,-0.050497,-0.831465,1.000000,-0.006626,-0.171733
heacy_rain_snow,-0.008097,-0.001942,-0.004074,-0.020065,-0.020238,0.013762,-0.004503,-0.015553,-0.006626,1.000000,-0.003218


In [ ]:
pd.plotting.scatter_matrix(df_slice3);

In [31]:
df_slice3.columns

Index(['datetime', 'Ausleihzahl', 'holiday', 'workingday', 'temp', 'atemp',
       'humidity', 'windspeed', 'clear_few_clouds', 'cloudy_mist',
       'heacy_rain_snow', 'light_snow_rain_thunderstorm'],
      dtype='object')

In [20]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [51]:
# Split data in Inputs (X) and outputs (Y)
X = df_slice3[['temp', 'atemp',
       'humidity', 'windspeed']]
# X = df[df.columns.difference(['y','article'])] # alle Spalten außer 'y' und 'article'
Y = df_slice3['Ausleihzahl'] 

In [52]:
# Scale X (to mean 0) and std 1 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=False) # with_mean=False: dont rescale the means to being all zero
Xs = pd.DataFrame(scaler.fit_transform(X[X.columns]))
Xs.columns = X.columns

In [53]:
regr = linear_model.LinearRegression()
regr.fit(Xs, Y)
pred = regr.predict(Xs)
print(Xs.columns.values)
print('Coefficients: \n', regr.coef_)
print('Intercept: \n', regr.intercept_)
print('On Training set:')
print("Mean squared error: %.2f" % mean_squared_error(Y, pred))
print("Variance of target variable: %.2f" % np.var(Y))
print('Variance explained: %.2f' % r2_score(Y, pred))

['temp' 'atemp' 'humidity' 'windspeed']
Coefficients: 
 [ 0.04156945  0.20839459 -0.22685055  0.00674661]
Intercept: 
 1.7375106229141426
On Training set:
Mean squared error: 1.90
Variance of target variable: 2.02
Variance explained: 0.06


In [54]:
import statsmodels.api as sm
# This linear model does not use a constant, i.e. only y = a1*x1 + ... + an*xn , and NOT y = a1*x1 + ... + an*xn + c,
# However, if one adds a column to X with only 1's, then the coefficient of this column has the same effect as the intercept c above
Xsdqe = sm.add_constant(Xs)

# Learn the "Ordinary Least Squares" model
est = sm.OLS(Y, Xsdqe) # Create the object and hand it the data
est = est.fit() # Let it compute the regression coefficients
print(est.summary()) # show the results - including all sorts of metrics

ValueError: The indices for endog and exog are not aligned

In [18]:
import scipy.stats.shapiro

ModuleNotFoundError: No module named 'scipy.stats.shapiro'

In [ ]:
def check_normality(data):
    test_stat_normality, p_value_normality=stats.shapiro(data)
    print("p value:%.4f" % p_value_normality)
    if p_value_normality <0.05:
        print("Reject null hypothesis >> The data is not normally distributed")
    else:
        print("Fail to reject null hypothesis >> The data is normally distributed")

In [63]:
df_sclice4= df_slice2[(df_slice2.holiday==1)&(df_slice2.windspeed>35)].copy()

In [70]:
df_slice5 = df_sclice4.groupby(pd.Grouper(key="datetime",freq="min")).agg(Ausleihzahl=("registered_customer","count"),holiday=("holiday","mean"),workingday=("workingday","mean"),temp=("temp","mean"),atemp=("atemp","mean"),humidity=("humidity","mean"),windspeed=("windspeed","mean"),clear_few_clouds=("weather_clear, few clouds","mean"),cloudy_mist=("weather_cloudy, mist","mean"),heacy_rain_snow=("weather_heacy rain or thunderstorm or snow or ice pallets","mean"),light_snow_rain_thunderstorm=("weather_light snow or rain or thunderstorm","mean")).reset_index()

In [71]:
#df_slice5[df_slice5.temp.isna()]
df_slice5.dropna(inplace=True)

In [72]:
df_slice5

,datetime,Ausleihzahl,holiday,workingday,temp,atemp,humidity,windspeed,clear_few_clouds,cloudy_mist,heacy_rain_snow,light_snow_rain_thunderstorm
0,2020-09-07 22:04:00,1,1.0,0.0,24.60,27.275,88.0,39.0007,0.0,1.0,0.0,0.0
1,2020-09-07 22:05:00,1,1.0,0.0,24.60,27.275,88.0,39.0007,0.0,1.0,0.0,0.0
2,2020-09-07 22:06:00,1,1.0,0.0,24.60,27.275,88.0,39.0007,0.0,1.0,0.0,0.0
3,2020-09-07 22:07:00,1,1.0,0.0,24.60,27.275,88.0,39.0007,0.0,1.0,0.0,0.0
17,2020-09-07 22:21:00,1,1.0,0.0,24.60,27.275,88.0,39.0007,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
160667,2020-12-28 11:51:00,1,1.0,0.0,13.94,14.395,46.0,39.0007,1.0,0.0,0.0,0.0
160669,2020-12-28 11:53:00,1,1.0,0.0,13.94,14.395,46.0,39.0007,1.0,0.0,0.0,0.0
160670,2020-12-28 11:54:00,1,1.0,0.0,13.94,14.395,46.0,39.0007,1.0,0.0,0.0,0.0
160673,2020-12-28 11:57:00,1,1.0,0.0,13.94,14.395,46.0,39.0007,1.0,0.0,0.0,0.0


In [73]:
# Split data in Inputs (X) and outputs (Y)
X = df_slice5[['temp', 'atemp',
       'humidity', 'windspeed']]
# X = df[df.columns.difference(['y','article'])] # alle Spalten außer 'y' und 'article'
Y = df_slice5['Ausleihzahl'] 

In [74]:
# Scale X (to mean 0) and std 1 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=False) # with_mean=False: dont rescale the means to being all zero
Xs = pd.DataFrame(scaler.fit_transform(X[X.columns]))
Xs.columns = X.columns

In [75]:
regr = linear_model.LinearRegression()
regr.fit(Xs, Y)
pred = regr.predict(Xs)
print(Xs.columns.values)
print('Coefficients: \n', regr.coef_)
print('Intercept: \n', regr.intercept_)
print('On Training set:')
print("Mean squared error: %.2f" % mean_squared_error(Y, pred))
print("Variance of target variable: %.2f" % np.var(Y))
print('Variance explained: %.2f' % r2_score(Y, pred))

['temp' 'atemp' 'humidity' 'windspeed']
Coefficients: 
 [-1.83833963  1.19194204  0.80104009 -0.12735124]
Intercept: 
 5.84455821957891
On Training set:
Mean squared error: 0.38
Variance of target variable: 0.40
Variance explained: 0.03
